# Sowing Success: How Machine Learning Helps Farmers Select the Best Crops

![Farmer in a field](farmer_in_a_field.jpg)

Measuring essential soil metrics such as nitrogen, phosphorous, potassium levels, and pH value is an important aspect of assessing soil condition. However, it can be an expensive and time-consuming process, which can cause farmers to prioritize which metrics to measure based on their budget constraints.

Farmers have various options when it comes to deciding which crop to plant each season. Their primary objective is to maximize the yield of their crops, taking into account different factors. One crucial factor that affects crop growth is the condition of the soil in the field, which can be assessed by measuring basic elements such as nitrogen and potassium levels. Each crop has an ideal soil condition that ensures optimal growth and maximum yield.

A farmer reached out to you as a machine learning expert for assistance in selecting the best crop for his field. They've provided you with a dataset called `soil_measures.csv`, which contains:

- `"N"`: Nitrogen content ratio in the soil
- `"P"`: Phosphorous content ratio in the soil
- `"K"`: Potassium content ratio in the soil
- `"pH"` value of the soil
- `"crop"`: categorical values that contain various crops (target variable).

Each row in this dataset represents various measures of the soil in a particular field. Based on these measurements, the crop specified in the `"crop"` column is the optimal choice for that field.  

In this project, you will build multi-class classification models to predict the type of `"crop"` and identify the single most importance feature for predictive performance.

In [79]:
# All required libraries are imported here for you.
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
# Load the dataset
crops = pd.read_csv("soil_measures.csv")

We confirm that the dataframe has been read correctly:

In [80]:
print(crops.head())

    N   P   K        ph  crop
0  90  42  43  6.502985  rice
1  85  58  41  7.038096  rice
2  60  55  44  7.840207  rice
3  74  35  40  6.980401  rice
4  78  42  42  7.628473  rice


Let's see how many missing values there are in each column:

In [81]:
print(crops.isna().sum())

N       0
P       0
K       0
ph      0
crop    0
dtype: int64


Now, let's see the type of data we have in each of the columns:

In [82]:
print(crops.dtypes)

N         int64
P         int64
K         int64
ph      float64
crop     object
dtype: object


Let's look at the unique values in our columns:

In [83]:
for column in crops.columns:
    print(f'Valores únicos en la columna {column}, son:')
    valores_unicos = crops[column].astype(str).unique() if crops[column].dtype == 'object' else crops[column].unique()
    print(valores_unicos)

Valores únicos en la columna N, son:
[ 90  85  60  74  78  69  94  89  68  91  93  77  88  76  67  83  98  66
  97  84  73  92  95  99  63  62  64  82  79  65  75  71  72  70  86  61
  81  80 100  87  96  40  23  39  22  36  32  58  59  42  28  43  27  50
  25  31  26  54  57  49  46  38  35  52  44  24  29  20  56  37  51  41
  34  30  33  47  53  45  48  13   2  17  12   6  10  19  11  18  21  16
   9   1   7   8   0   3   4   5  14  15  55 105 108 118 101 106 109 117
 114 110 112 111 102 116 119 107 104 103 120 113 115 133 136 126 121 129
 122 140 131 135 123 125 139 132 127 130 134]
Valores únicos en la columna P, son:
[ 42  58  55  35  37  53  54  46  56  50  48  38  45  40  59  41  47  49
  51  57  39  43  44  60  52  36  72  67  73  70  62  74  66  63  71  78
  80  68  65  77  76  79  61  64  69  75  24  18  26  27  25  21  30  11
   5  10   7  20  22  15  23   8  16  29  17   6  19  13   9  14  28  94
  95  92  89  88  87  85  86  83  91  81  84  90  82  93  33  31  34  32
 130

Let's do a quick overview of the statistics on each of the numerical columns:

In [84]:
crops.describe()

,N,P,K,ph
count,2200.000000,2200.000000,2200.000000,2200.000000
mean,50.551818,53.362727,48.149091,6.469480
std,36.917334,32.985883,50.647931,0.773938
min,0.000000,5.000000,5.000000,3.504752
25%,21.000000,28.000000,20.000000,5.971693
50%,37.000000,51.000000,32.000000,6.425045
75%,84.250000,68.000000,49.000000,6.923643
max,140.000000,145.000000,205.000000,9.935091


Now we will make a StandarScaler for the numeric columns:

In [85]:
numeric_columns = crops.select_dtypes(include=['number']).columns.tolist()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(crops[numeric_columns])
X_scaled_df = pd.DataFrame(X_scaled, columns=numeric_columns, index=crops.index)

We declare the target variable:

In [86]:
y = crops['crop'].values

We split the train and test sets:

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.2, random_state=42)

Now we build a dictionary with the feature and its f1 score:

In [88]:
features_dict = {}

for feature in ["N", "P", "K", "ph"]:
    log_reg = LogisticRegression(multi_class="multinomial", max_iter=1000)

    X_train_feature = X_train[[feature]]
    X_test_feature = X_test[[feature]]

    log_reg.fit(X_train_feature, y_train)

    y_pred = log_reg.predict(X_test_feature)

    f1 = metrics.f1_score(y_test, y_pred, average="weighted") 
    print(f"F1 Score para {feature}: {f1}")

    features_dict[feature] = f1

F1 Score para N: 0.10689900116508289
F1 Score para P: 0.08426955444720076
F1 Score para K: 0.13831456375684123
F1 Score para ph: 0.045464856528065166


Finally, we build a dictionary with the best values:

In [89]:
best_feature = max(features_dict, key=features_dict.get)

best_predictive_feature = {best_feature: features_dict[best_feature]}

print("Mejor feature predictiva:", best_predictive_feature)

Mejor feature predictiva: {'K': 0.13831456375684123}
